## Goal:

<insert Kelli's message from slack here>

## Imports

In [25]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
import numpy as np

## Data read-in

In [26]:
df = pd.read_csv('CHI_agents_data-Table 1.csv',low_memory=False)

In [27]:
# Function to convert WKT format to a Point geometry
def to_point(coord):
    try:
        if isinstance(coord, str) and 'POINT' in coord:
            # Use WKT to convert the string into a Point object
            return wkt.loads(coord)
        else:
            return None  # Return None for invalid or missing coordinates
    except Exception as e:
        return None  # Catch any issues

# Apply the conversion to the 'geometry' column
df['geometry'] = df['geometry'].apply(to_point)

# Now convert the DataFrame to a GeoDataFrame
df_geo = gpd.GeoDataFrame(df, geometry='geometry')

In [28]:
boundaries_df = gpd.read_file('Boundaries - Community Areas (current).geojson')

In [66]:
communities_to_keep_old = [
    'NEAR SOUTH SIDE',
    'ARMOUR SQUARE',
    'BRIDGEPORT',
    'MCKINLEY PARK',
    'BRIGHTON PARK',
    'ARCHER HEIGHTS',
    'GARFIELD RIDGE',
    'CLEARING',
    'WEST ELSDON',
    'GAGE PARK',
    'NEW CITY',
    'FULLER PARK',
    'GRAND BOULEVARD',
    'DOUGLAS',
    'OAKLAND',
    'KENWOOD',
    'HYDE PARK',
    'WASHINGTON PARK',
    'ENGLEWOOD',
    'WEST ENGLEWOOD',
    'CHICAGO LAWN',
    'WEST LAWN',
    'WOODLAWN',
    'GREATER GRAND CROSSING',
    'SOUTH SHORE',
    'AUBURN GRESHAM',
    'ASHBURN',
    'CHATHAM',
    'AVALON PARK',
    'SOUTH CHICAGO',
    'CALUMET HEIGHTS',
    'BURNSIDE',
    'WASHINGTON HEIGHTS',
    'BEVERLY',
    'MOUNT GREENWOOD',
    'MORGAN PARK',
    'WEST PULLMAN',
    'SOUTH DEERING',
    'RIVERDALE',
    'EAST SIDE',
    'HEGEWISCH'
]

communities_to_keep = [
    'NEAR SOUTH SIDE',
    'ARMOUR SQUARE',
    'BRIDGEPORT',
    'MCKINLEY PARK',
    'BRIGHTON PARK',
    'ARCHER HEIGHTS',
    'GARFIELD RIDGE',
    'CLEARING',
    'WEST ELSDON',
    'GAGE PARK',
    'NEW CITY',
    'FULLER PARK',
    'GRAND BOULEVARD',
    'DOUGLAS',
    'OAKLAND',
    'KENWOOD',
    'HYDE PARK',
    'WASHINGTON PARK',
    'ENGLEWOOD',
    'WEST ENGLEWOOD',
    'CHICAGO LAWN',
    'WEST LAWN',
    'WOODLAWN',
    'GREATER GRAND CROSSING',
    'SOUTH SHORE',
    'AUBURN GRESHAM',
    'ASHBURN',
    'CHATHAM',
    'AVALON PARK',
    'SOUTH CHICAGO',
    'CALUMET HEIGHTS',
    'BURNSIDE',
    'WASHINGTON HEIGHTS',
    'BEVERLY',
    'MOUNT GREENWOOD',
    'MORGAN PARK',
    'PULLMAN',
    'WEST PULLMAN',
    'ROSELAND',
    'SOUTH DEERING',
    'RIVERDALE',
    'EAST SIDE',
    'HEGEWISCH'
]

In [67]:
set1 = set(communities_to_keep_old)
set2 = set(communities_to_keep)

# Elements in list1 but not in list2
not_in_list2 = list(set1 - set2)
print(f"Elements in list1 but not in list2: {not_in_list2}")

# Elements in list2 but not in list1
not_in_list1 = list(set2 - set1)
print(f"Elements in list2 but not in list1: {not_in_list1}")

Elements in list1 but not in list2: []
Elements in list2 but not in list1: ['ROSELAND', 'PULLMAN']


In [30]:
filtered_boundaries = boundaries_df[boundaries_df['community'].isin(communities_to_keep)]

In [31]:
df_geo = df_geo.set_crs(4326)

In [32]:
# Read in neighborhood boundaries
neighborhood_boundaries = filtered_boundaries

# Clean and convert the 'Price' column to integers
df_geo['Price_int'] = df_geo['Price'].str.replace('[\$,]', '', regex=True).astype(int)

# Ensure both GeoDataFrames (df_geo and neighborhood_boundaries) have the same CRS
if df_geo.crs != neighborhood_boundaries.crs:
    df_geo = df_geo.to_crs(neighborhood_boundaries.crs)  # Transform CRS to match

# Perform a spatial join to find which points (brokerages) fall within which community
joined_gdf = gpd.sjoin(df_geo, neighborhood_boundaries, how="inner", predicate='within')

# # Group by community and agent/team to calculate counts and total sales
# grouped = joined_gdf.groupby(['community', 'Final_Agent/Team']).agg(
#     count=('Final_Agent/Team', 'count'),
#     total_price=('Price_int', 'sum')
# ).reset_index().nlargest(20)

# # Find the agent/team with the highest count for each community
# max_count_idx = grouped.groupby("community")['count'].idxmax()
# agent_result = grouped.loc[max_count_idx, ['community', 'Final_Agent/Team', 'count', 'total_price']]
# agent_result = agent_result.rename(columns={'count': 'agent_count', 'total_price': 'agent_price'})

# # Calculate community-level totals
# community_result = grouped.groupby('community').agg(
#     community_count=('count', 'sum'),
#     community_price=('total_price', 'sum')
# ).reset_index()

# # Merge agent results with community-level totals
# merged_df = pd.merge(agent_result, community_result, on='community', how='inner')

# # Drop unnecessary columns from neighborhood boundaries
# columns_to_drop = ['area', 'shape_area', 'perimeter', 'area_num_1', 'area_numbe', 'comarea_id', 'comarea', 'shape_len']
# neighborhood_boundaries = neighborhood_boundaries.drop(columns=columns_to_drop, errors='ignore')

# # Merge with neighborhood boundaries
# final_merge = neighborhood_boundaries.merge(merged_df, on='community')
# final_merge = final_merge.rename(columns={'Final_Agent/Team': 'final_agent_team'})

# # Save the result to a GeoJSON file
# final_merge.to_file("final_merge.geojson", driver='GeoJSON')


In [33]:
joined_gdf

,mls_id,Address,Location,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,Brokerage,...,index_right,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len
6,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",buy_agent_1,buyer,Anne Fan,Anne Fan,Compass,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
7,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",list_agent_1,listing,George Toscas,George Toscas,ACO Commercial,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
8,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",list_agent_2,listing,Linda Hattar,Linda Hattar,ACO Commercial,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
126,11993747,4825 S Woodlawn Ave,CHI - Kenwood,60615.0,"$4,000,000",list_agent_1,listing,Eugene Fu,Eugene Fu,@properties Christie's International Real Estate,...,4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062
135,11993747,4825 S Woodlawn Ave,CHI - Kenwood,60615.0,"$4,000,000",buy_agent_1,buyer,Susan O'Connor,Susan O'Connor,Berkshire Hathaway HomeServices Chicago,...,4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108203,11836116,12910 S Parnell Ave,CHI - West Pullman,60628.0,"$100,000",list_agent_1,listing,Vera Brown,Vera Brown,R G Ramsey & Associates,...,51,WEST PULLMAN,0,99365198.0822,0,53,53,0,0,50023.8430008
108204,12041412,8222 S Perry Ave,CHI - Chatham,60620.0,"$100,000",list_agent_1,listing,Vernon Lilly,Vernon Lilly,EXIT True Design Realty LLC,...,39,CHATHAM,0,82320670.3112,0,44,44,0,0,42006.9450094
108208,11887086,7558 S Marshfield Ave,CHI - Auburn Gresham,60620.0,"$100,000",list_agent_1,listing,Victor Vita,Victor Vita,Vylla Home,...,69,AUBURN GRESHAM,0,105065353.602,0,71,71,0,0,46757.7217161
108210,11876519,6950 S Vernon Ave,CHI - Greater Grand Crossing,60637.0,"$100,000",list_agent_1,listing,William Bates Jr,William Bates Jr,EXIT Strategy Realty,...,66,GREATER GRAND CROSSING,0,98853167.7093,0,69,69,0,0,54645.3302996


In [34]:
top_agents_list = joined_gdf.value_counts('Final_Agent/Team').head(31)
top_agents_list = top_agents_list.keys().to_list()

In [35]:
joined_gdf.columns

Index(['mls_id', 'Address', 'Location', 'Zip_Code', 'Price', 'agent_no',
       'side', 'agent_name', 'Final_Agent/Team', 'Brokerage', 'url',
       'TRD_Note', 'geometry', 'SqFt', 'price_per_sq_ft', 'year_built',
       'broker_email', 'data_source_name', 'within_county', 'Price_int',
       'index_right', 'community', 'area', 'shape_area', 'perimeter',
       'area_num_1', 'area_numbe', 'comarea_id', 'comarea', 'shape_len'],
      dtype='object')

In [36]:
gdf_for_export = joined_gdf[['mls_id', 'Address', 'Location', 'Zip_Code', 'Price', 'agent_no',
       'side', 'agent_name', 'Final_Agent/Team', 'Brokerage', 'url',
       'TRD_Note', 'geometry', 'SqFt', 'price_per_sq_ft', 'year_built',
       'broker_email']]

In [37]:
# Create an ExcelWriter instance
with pd.ExcelWriter('all_agents_data_extra_geoms.xlsx') as writer:
    for agent in top_agents_list:
        # Filter for the current agent
        x = gdf_for_export[gdf_for_export['Final_Agent/Team'] == agent]
        
        # Write each agent's dataframe to a different sheet
        x.to_excel(writer, sheet_name=agent, index=False)


In [38]:
joined_gdf

,mls_id,Address,Location,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,Brokerage,...,index_right,community,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len
6,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",buy_agent_1,buyer,Anne Fan,Anne Fan,Compass,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
7,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",list_agent_1,listing,George Toscas,George Toscas,ACO Commercial,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
8,11284349,3000 S Pitney Ct,CHI - Bridgeport,60608.0,"$8,000,000",list_agent_2,listing,Linda Hattar,Linda Hattar,ACO Commercial,...,57,BRIDGEPORT,0,58291519.2767,0,60,60,0,0,32732.7183268
126,11993747,4825 S Woodlawn Ave,CHI - Kenwood,60615.0,"$4,000,000",list_agent_1,listing,Eugene Fu,Eugene Fu,@properties Christie's International Real Estate,...,4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062
135,11993747,4825 S Woodlawn Ave,CHI - Kenwood,60615.0,"$4,000,000",buy_agent_1,buyer,Susan O'Connor,Susan O'Connor,Berkshire Hathaway HomeServices Chicago,...,4,KENWOOD,0,29071741.9283,0,39,39,0,0,23325.1679062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108203,11836116,12910 S Parnell Ave,CHI - West Pullman,60628.0,"$100,000",list_agent_1,listing,Vera Brown,Vera Brown,R G Ramsey & Associates,...,51,WEST PULLMAN,0,99365198.0822,0,53,53,0,0,50023.8430008
108204,12041412,8222 S Perry Ave,CHI - Chatham,60620.0,"$100,000",list_agent_1,listing,Vernon Lilly,Vernon Lilly,EXIT True Design Realty LLC,...,39,CHATHAM,0,82320670.3112,0,44,44,0,0,42006.9450094
108208,11887086,7558 S Marshfield Ave,CHI - Auburn Gresham,60620.0,"$100,000",list_agent_1,listing,Victor Vita,Victor Vita,Vylla Home,...,69,AUBURN GRESHAM,0,105065353.602,0,71,71,0,0,46757.7217161
108210,11876519,6950 S Vernon Ave,CHI - Greater Grand Crossing,60637.0,"$100,000",list_agent_1,listing,William Bates Jr,William Bates Jr,EXIT Strategy Realty,...,66,GREATER GRAND CROSSING,0,98853167.7093,0,69,69,0,0,54645.3302996


In [39]:
joined_gdf['price_as_int'] = joined_gdf['Price'].str.replace("$","",regex=False)
joined_gdf['price_as_int'] = joined_gdf['price_as_int'].str.replace(",","",regex=False).astype(int)

In [40]:
agents_by_transaction = joined_gdf.groupby('Final_Agent/Team')['price_as_int'].sum()

In [41]:
joined_gdf.columns

Index(['mls_id', 'Address', 'Location', 'Zip_Code', 'Price', 'agent_no',
       'side', 'agent_name', 'Final_Agent/Team', 'Brokerage', 'url',
       'TRD_Note', 'geometry', 'SqFt', 'price_per_sq_ft', 'year_built',
       'broker_email', 'data_source_name', 'within_county', 'Price_int',
       'index_right', 'community', 'area', 'shape_area', 'perimeter',
       'area_num_1', 'area_numbe', 'comarea_id', 'comarea', 'shape_len',
       'price_as_int'],
      dtype='object')

In [42]:
joined_gdf.groupby('Final_Agent/Team')['price_as_int'].sum()

Final_Agent/Team
Aaqila Harvey       569900
Aaron Campbell      127000
Aaron Franklin      252000
Aaron Gaines       5065147
Aaron Greenberg     455000
                    ...   
Zoe Wohlsifer       135000
Zoey Zhu           2459588
Zofia Strzep       1944800
Zofia Zon-Leiva     620100
Zorica Ledic        268000
Name: price_as_int, Length: 5430, dtype: int64

In [43]:
agents_by_transaction.sort_values(ascending=False).to_csv("agents_by_dollar_volume_extra_geoms.csv")

In [44]:
top_199_agents_by_dollar_volume = agents_by_transaction.sort_values(ascending=False).head(40).keys().to_list()

In [45]:
# Create an ExcelWriter instance
with pd.ExcelWriter('all_agents_dollar_data_extra_geoms.xlsx') as writer:
    for agent in top_199_agents_by_dollar_volume:
        # Filter for the current agent
        x = gdf_for_export[gdf_for_export['Final_Agent/Team'] == agent]
        
        # Write each agent's dataframe to a different sheet
        x.to_excel(writer, sheet_name=agent, index=False)


## Collect property types

In [46]:
joined_gdf['api_url'] = joined_gdf['url'].str[22:]

In [47]:
joined_gdf['api_url']

6         /IL/Chicago/3000-S-Pitney-Ct-60608/home/177573811
7         /IL/Chicago/3000-S-Pitney-Ct-60608/home/177573811
8         /IL/Chicago/3000-S-Pitney-Ct-60608/home/177573811
126       /IL/Chicago/4825-S-Woodlawn-Ave-60615/home/139...
135       /IL/Chicago/4825-S-Woodlawn-Ave-60615/home/139...
                                ...                        
108203    /IL/Chicago/12910-S-Parnell-Ave-60628/home/130...
108204     /IL/Chicago/8222-S-Perry-Ave-60620/home/13225737
108208    /IL/Chicago/7558-S-Marshfield-Ave-60620/home/1...
108210    /IL/Chicago/6950-S-Vernon-Ave-60637/unit-2/hom...
108211    /IL/Chicago/9549-S-Bensley-Ave-60617/home/1304...
Name: api_url, Length: 14338, dtype: object

In [63]:
x = joined_gdf[joined_gdf['Location'] == 'CHI - Roseland']
y = joined_gdf[joined_gdf['Location'] == 'CHI - Pullman']

In [73]:
# x[x['Final_Agent/Team'] == 'Frank Montro']
# x['Final_Agent/Team'].value_counts()
y['Final_Agent/Team'].value_counts()

Final_Agent/Team
Leslie Smith          3
Michael Wolski        3
Linda Maguire         2
Constantin Lupancu    2
Tanesha Pratt         2
                     ..
Gwendolyn Robinson    1
Eleazar Vazquez       1
Khalilah Stokes       1
Dorian Johnson        1
Samuel Granville      1
Name: count, Length: 97, dtype: int64

In [75]:
y[y['Final_Agent/Team'] == 'Frank Montro'].to_csv("Montro_pullman_sales.csv")

In [74]:
y.groupby('Final_Agent/Team')['price_as_int'].sum().sort_values(ascending=False).head(60)

Final_Agent/Team
Michael Wolski               747000
Linda Maguire                649900
Leslie Smith                 596500
Constantin Lupancu           522000
Katharine Sanderson          509700
Cassandra Bockman Hart       439000
Tanesha Pratt                420000
Sandra Lee                   395000
Carlton McLendon             365000
Trudy Holmes                 365000
Kywana Ross                  355000
Yolanda Edwards              339000
Gustavo Ruvalcaba            339000
Non Member                   324400
Michael Olszewski            287000
Anthony Kirkland             265000
Frank Montro                 264000
Ken Yu                       264000
Dalton Garcia                264000
Foster Robinson              259900
Angela Walker                259900
Michael LaFargue             259700
Nicholas Hansen              258000
Kathy Szuba                  258000
Kimberly Adams               257000
Patricia Sanchez             254900
Pamela Thomas                250000
Dejeanette 

In [49]:
to_be_sliced = joined_gdf.reset_index(drop=True)

In [39]:
group_1 = to_be_sliced.iloc[0:8000]
group_2 = to_be_sliced.iloc[8000:]

In [27]:
# import requests

# url = "https://redfin-com-data.p.rapidapi.com/properties/details"

# # querystring = {"url":"/IL/Chicago/6605-S-Kimbark-Ave-60637/home/12570870"}

# querystring = {"url":"/IL/Chicago/100-E-14th-St-60605/unit-1112/home/39569782"}

# headers = {
#     "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
#     "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
# }

# response = requests.get(url, headers=headers, params=querystring)

In [28]:
# response.json()['data']['belowTheFold']['publicRecordsInfo']['basicInfo']['propertyTypeName']

In [29]:
# def get_property_type(api_url=str):
    
#     querystring = {"url":api_url}

#     headers = {
#         "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
#         "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
#     }

#     response = requests.get(url, headers=headers, params=querystring)
    
#     property_type = response.json()['data']['belowTheFold']['publicRecordsInfo']['basicInfo']['propertyTypeName']
    
#     return property_type

In [40]:
import time
import requests

def get_property_type_with_error(api_url, max_retries=3, backoff_factor=1.0):
    """
    Fetch property type from the Redfin API, with error handling and exponential backoff.

    Returns:
      - The property type (string), OR
      - An error message (string) if something goes wrong.
    """
    
    url = "https://redfin-com-data.p.rapidapi.com/properties/details"
    
    querystring = {"url": api_url}
    headers = {
        "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",  
        "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
    }

    attempt = 1
    while attempt <= max_retries:
        try:
            response = requests.get(url, headers=headers, params=querystring, timeout=10)
            response.raise_for_status()

            data = response.json()
            property_type = data['data']['belowTheFold']['publicRecordsInfo']['basicInfo']['propertyTypeName']
            
            return property_type
        
        except requests.exceptions.RequestException as re:
            # Networking or HTTP error
            if attempt == max_retries:
                # Return error message as a string
                return f"ERROR: Request failed after {max_retries} retries. {re}"
            else:
                # Exponential backoff
                delay = backoff_factor * (2 ** (attempt - 1))
                time.sleep(delay)
                attempt += 1

        except KeyError as ke:
            # JSON doesn't have the fields we expect
            return f"ERROR: Unexpected JSON structure. Missing key: {ke}"

        except Exception as e:
            # Catch-all
            return f"ERROR: An unexpected error occurred: {e}"

In [43]:
from tqdm.auto import tqdm  # or from tqdm import tqdm
tqdm.pandas()              # This adds "progress_apply" to pandas DataFrames

In [ ]:
test_df = joined_gdf.tail(30) 

test_df['property_type'] = test_df['api_url'].progress_apply(get_property_type_with_error)

In [47]:
group_1['property_type'] = group_1['api_url'].progress_apply(get_property_type_with_error)
group_1.to_csv("group_1.csv")


  0%|          | 0/8000 [00:00<?, ?it/s]

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [48]:
group_2['property_type'] = group_2['api_url'].progress_apply(get_property_type_with_error)
group_2.to_csv("group_2.csv")

  0%|          | 0/5571 [00:00<?, ?it/s]

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [49]:
group_1['property_type'].iloc[0]

'Vacant Land'

In [50]:
group_1['property_type'].value_counts()

property_type
Single Family Residential                                        4013
Condo/Co-op                                                      1589
Multi-Family (2-4 Unit)                                          1367
Multi-Family (5+ Unit)                                            424
Townhouse                                                         408
Vacant Land                                                       103
ERROR: Unexpected JSON structure. Missing key: 'belowTheFold'      61
Other                                                              23
Parking                                                            12
Name: count, dtype: int64

In [51]:
group_2['property_type'].value_counts()

property_type
Single Family Residential                                                      3330
Condo/Co-op                                                                    1194
Multi-Family (2-4 Unit)                                                         718
Townhouse                                                                       129
Multi-Family (5+ Unit)                                                          127
Vacant Land                                                                      44
ERROR: Unexpected JSON structure. Missing key: 'belowTheFold'                    17
Parking                                                                           6
Other                                                                             4
ERROR: An unexpected error occurred: 'NoneType' object is not subscriptable       2
Name: count, dtype: int64

In [52]:
df_concat_property_types = pd.concat([group_1,group_2])

In [53]:
df_concat_property_types['property_type'].value_counts()

property_type
Single Family Residential                                                      7343
Condo/Co-op                                                                    2783
Multi-Family (2-4 Unit)                                                        2085
Multi-Family (5+ Unit)                                                          551
Townhouse                                                                       537
Vacant Land                                                                     147
ERROR: Unexpected JSON structure. Missing key: 'belowTheFold'                    78
Other                                                                            27
Parking                                                                          18
ERROR: An unexpected error occurred: 'NoneType' object is not subscriptable       2
Name: count, dtype: int64

In [59]:
# df_concat_property_types[df_concat_property_types['property_type'] == "ERROR: Unexpected JSON structure. Missing key: 'belowTheFold'"]['url'].iloc[0]

In [60]:
keep_list = [
    "Single Family Residential",
    "Condo/Co-op",
    "Townhouse"
]

In [61]:
final_df = df_concat_property_types[df_concat_property_types['property_type'].isin(keep_list)]

In [70]:
final_agent_list = final_df.groupby('Final_Agent/Team')['price_as_int'].sum().sort_values(ascending=False)

In [71]:
final_agent_list

Final_Agent/Team
Nadine Ferrata     28709300
Robert Sullivan    20029500
Susan O'Connor     14776500
Non Member         14423336
The Laricy Team    14276500
                     ...   
Darraneika Lacy      100000
Dorothy Turner       100000
Angela Upton         100000
Willie Jones         100000
Erica Smith          100000
Name: price_as_int, Length: 4612, dtype: int64

In [72]:
final_agent_list.to_csv("final_agent_list_transaction_volume_with_property_filtered.csv")

In [100]:
team_names_df = pd.read_csv("team_names.csv",usecols=["For_team_matching"])

In [101]:
team_names_df = team_names_df.dropna()

In [113]:
matched_team_df = final_df

In [114]:
# -----------------------------------------------------------------------------
# STEP 1: CREATE LOOKUP DICTIONARIES
# -----------------------------------------------------------------------------
def normalize_name(name: str) -> str:
    """Lowercase and strip whitespace to handle case mismatch."""
    return name.strip().lower()

# name_to_team: agent_name (lowercased) -> Official Team Name (unmodified)
name_to_team = {}

# team_to_members: official_team_name (lowercased) -> list of all team members (unmodified)
team_to_members = {}

for row in team_names_df.itertuples(index=False):
    # e.g.: "Alice Smith, Bob Johnson, Charlie Brown"
    members_raw = row.For_team_matching.split(",")
    
    # Clean up whitespace in each name, but don't lose original capitalization
    members = [m.strip() for m in members_raw]
    
    # We'll treat the first name as the "official team name"
    official_team_name = members[0]            # e.g. "Alice Smith"
    official_team_key  = normalize_name(official_team_name)  # e.g. "alice smith"
    
    # Save the full list of members for this team
    team_to_members[official_team_key] = members
    
    # Map each member to the same official team name
    for member in members:
        name_to_team[normalize_name(member)] = official_team_name

# -----------------------------------------------------------------------------
# STEP 2: UPDATE THE "final_df" COLUMNS
# -----------------------------------------------------------------------------
def get_team_name_or_original(agent_name: str, current_value: str) -> str:
    """Return the official team name if the agent belongs to a team, else original value."""
    agent_key = normalize_name(agent_name)
    if agent_key in name_to_team:
        # Overwrite with the official team name
        return name_to_team[agent_key]
    else:
        # Leave the existing name alone
        return current_value

def get_team_members(agent_name: str) -> str:
    """Return a comma-separated string of all members if agent belongs to a team, else None."""
    agent_key = normalize_name(agent_name)
    if agent_key in name_to_team:
        # Get the official team name
        official_team_name = name_to_team[agent_key]
        official_team_key  = normalize_name(official_team_name)
        # Convert the list of members to a single string
        all_members = team_to_members[official_team_key]
        return ", ".join(all_members)
    return None

# Create/update columns in final_df
matched_team_df["Final_Agent/Team"] = matched_team_df.apply(
    lambda row: get_team_name_or_original(
        agent_name=row["agent_name"],
        current_value=row["Final_Agent/Team"]
    ),
    axis=1
)

# Add a new column that shows all team members for that agent (or None if not on a team)
matched_team_df["Team_Members"] = matched_team_df["agent_name"].apply(get_team_members)

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [116]:
matched_final_agent_list = matched_team_df.groupby('Final_Agent/Team')['price_as_int'].sum().sort_values(ascending=False)



In [121]:
matched_final_agent_list.to_csv("matched_final_agent_list.csv")

In [128]:
matched_team_df.to_csv("matched_team_df.csv")

In [135]:
matched_team_df['mls_id'] = matched_team_df['mls_id'].astype(str)

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [138]:
matched_team_df['mls_id'].value_counts().head(60)

mls_id
11896882    4
11893822    4
12030043    4
11956458    4
11866009    4
11828670    4
11840285    4
12051913    4
11938887    4
12045369    4
11975552    4
11846499    4
12004485    4
11828021    4
11898153    4
11874078    4
11751869    4
11816920    4
11985298    4
12028301    4
11867772    4
11828915    4
11859206    4
11829674    4
11905981    4
11857372    4
12014016    4
11855029    4
11994702    4
11831574    4
12017476    4
11881460    4
11988158    4
11940428    4
11867069    4
11889710    4
11975668    4
11915224    4
11997805    4
11937939    4
11987310    4
11958865    4
11811315    4
11794014    4
11812398    4
11962182    4
11981676    4
12013605    4
12021378    4
12059763    4
11448896    4
12039492    4
11807395    4
11938631    4
11954947    4
12030082    3
11823690    3
11819124    3
11871232    3
11741064    3
Name: count, dtype: int64

In [140]:
matched_team_df[matched_team_df['mls_id'] == "12030043"]

,mls_id,Address,Location,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,Brokerage,...,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,price_as_int,api_url,property_type,Team_Members
6884,12030043,13025 S Manistee Ave,CHI - Hegewisch,60633.0,"$274,000",list_agent_2,listing,Hugo Gracia,Hugo Gracia,Compass,...,0,55,55,0,0,73692.3821322,274000,/IL/Chicago/13025-S-Manistee-Ave-60633/home/13...,Single Family Residential,None
6886,12030043,13025 S Manistee Ave,CHI - Hegewisch,60633.0,"$274,000",list_agent_1,listing,Leslie Cifuentes,Leslie Cifuentes,Compass,...,0,55,55,0,0,73692.3821322,274000,/IL/Chicago/13025-S-Manistee-Ave-60633/home/13...,Single Family Residential,None
6887,12030043,13025 S Manistee Ave,CHI - Hegewisch,60633.0,"$274,000",buy_agent_2,buyer,Lourdes Fernando,Lourdes Fernando,Keller Williams ONEChicago,...,0,55,55,0,0,73692.3821322,274000,/IL/Chicago/13025-S-Manistee-Ave-60633/home/13...,Single Family Residential,None
6888,12030043,13025 S Manistee Ave,CHI - Hegewisch,60633.0,"$274,000",buy_agent_1,buyer,Tiffany Bennett,Tiffany Bennett,Keller Williams ONEChicago,...,0,55,55,0,0,73692.3821322,274000,/IL/Chicago/13025-S-Manistee-Ave-60633/home/13...,Single Family Residential,None


## Create separate CSVs for top 30 and write into workbook

In [36]:
import pandas as pd

In [41]:
teams = pd.read_clipboard(sep=',')

In [14]:
dealsheet = pd.read_csv("ChicagoSouthSideAgentsTeamsMatched - deal_sheet.csv")

In [42]:
teams_list = teams['Final_Agent/Team'].to_list()

In [44]:
dealsheet.columns

Index(['mls_id', 'Address', 'Location', 'property_type', 'Zip_Code', 'Price',
       'agent_no', 'side', 'agent_name', 'Final_Agent/Team', 'Team_Members',
       'Brokerage', 'url', 'TRD_Note', 'geometry', 'SqFt', 'price_per_sq_ft',
       'year_built', 'broker_email', 'data_source_name', 'within_county',
       'Price_int', 'index_right', 'community', 'area', 'shape_area',
       'perimeter', 'area_num_1', 'area_numbe', 'comarea_id', 'comarea',
       'shape_len', 'price_as_int', 'api_url'],
      dtype='object')

In [45]:
# import pandas as pd

# # Create a DataFrame
# df = pd.DataFrame({
#     'Name': ['Alice', 'Bob', 'Charlie', 'David'],
#     'Age': [25, 30, 22, 28]
# })

# # List of names to filter by
# names_to_filter = ['Alice', 'Charlie']

# # Filter the DataFrame
# filtered_df = df[df['Name'].isin(names_to_filter)]

# print(filtered_df)

filtered_dealsheet = dealsheet[dealsheet['Final_Agent/Team'].isin(teams_list)]

In [46]:
filtered_dealsheet['Final_Agent/Team'].value_counts()

Final_Agent/Team
Frank Montro               55
Non Member                 51
Naja Morris                48
Nadine Ferrata             42
QianKun Chen               35
Pablo Galarza              32
Robert Sullivan            32
Salvador Gonzalez          30
Sybil Martin               30
The Laricy Team            29
Carlos Sanchez             28
Lynda Sanchez-Werner       26
Maria Castillo             26
Grigory Pekarsky           26
Lorena Ramirez-Carrillo    23
Kellye Jackson             22
Joanne Bullen              22
Jose Aguilar               21
Erika Villegas             20
Qizhong Guan               20
John Healy                 19
Jennifer Liu               18
Meghan Healy               18
Jacob Reiner               16
Benjamin Lissner           16
Lane Chesebro              15
Scott Berg                 15
Nancy Hotchkiss            14
Cory Tanzer                14
Sheila Dantzler            13
Robert Yoshimura           13
Stefanie D'Agostino        13
Ben Lalez Team         

In [47]:
# Suppose your dataframe is 'filtered_dealsheet'
# and the column of interest is "Final_Agent/Team"

# 1. Get all unique agents/teams
unique_agents = filtered_dealsheet["Final_Agent/Team"].unique()

# 2. Create a new Excel file and write each subset to a different sheet
output_file = "output_file.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    for agent in unique_agents:
        # 3. Filter rows for the current agent/team
        agent_data = filtered_dealsheet[filtered_dealsheet["Final_Agent/Team"] == agent]
        
        # 4. Write to a sheet named after the agent/team
        #    Ensure sheet_name does not exceed Excel’s 31-char limit
        sheet_name = str(agent)[:31]
        
        agent_data.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Workbook created: {output_file}")


Workbook created: output_file.xlsx


In [50]:
filtered_dealsheet['property_type'].value_counts()

property_type
Single Family Residential    514
Condo/Co-op                  273
Townhouse                     94
Name: count, dtype: int64

## Team Reconciliation 

### MVP Team

https://mvpteamchicago.com/associates

In [106]:
dealsheet[dealsheet['agent_name'].str.contains("Knot")]

,mls_id,Address,Location,property_type,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,...,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,price_as_int,api_url
73,12048790,5428 S Dorchester Ave,CHI - Hyde Park,Single Family Residential,60615,"$1,100,000",list_agent_2,listing,Albert Knott,Albert Knott,...,0,45105380.17,0,41,41,0,0,29746.7082,1100000,/IL/Chicago/5428-S-Dorchester-Ave-60615/home/1...


In [111]:
dealsheet[dealsheet['agent_name'].str.contains("Strumberger")]

,mls_id,Address,Location,property_type,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,...,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,price_as_int,api_url


In [131]:
dealsheet[dealsheet['agent_name'].str.contains("Mariduena|Giglio|Knot|Strumberger")]['price_as_int'].sum()

8387000

In [132]:
mvp_team = dealsheet[dealsheet['agent_name'].str.contains("Mariduena|Giglio|Knot|Strumberger")]
mvp_team.to_csv("MVP_team.csv")

In [114]:
dealsheet[dealsheet['agent_name'].str.contains("Calabrese")]

,mls_id,Address,Location,property_type,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,...,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,price_as_int,api_url


In [118]:
dealsheet[dealsheet['agent_name'].str.contains("Giglio")]

,mls_id,Address,Location,property_type,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,...,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,price_as_int,api_url
4,11831344,1801-1811 S Prairie Ave,Prairie District Homes,Single Family Residential,60616,"$3,900,000",list_agent_1,listing,Melanie Giglio,Melanie Giglio,...,0,49769639.45,0,33,33,0,0,45053.16510,3900000,/IL/Chicago/1801-S-Prairie-Ave-60616/home/2281...
75,12048790,5428 S Dorchester Ave,CHI - Hyde Park,Single Family Residential,60615,"$1,100,000",list_agent_1,listing,Melanie Giglio,Melanie Giglio,...,0,45105380.17,0,41,41,0,0,29746.70820,1100000,/IL/Chicago/5428-S-Dorchester-Ave-60615/home/1...
192,12029948,4922-P S Cornell Ave,Cornell Square,Single Family Residential,60615,"$845,000",list_agent_1,listing,Melanie Giglio,Melanie Giglio,...,0,29071741.93,0,39,39,0,0,23325.16791,845000,/IL/Chicago/4922-S-Cornell-Ave-60615/unit-P/ho...
4001,11823151,1255 S State St #1617,CHI - Near South Side,Condo/Co-op,60691,"$300,000",buy_agent_1,buyer,Melanie Giglio,Melanie Giglio,...,0,49769639.45,0,33,33,0,0,45053.16510,300000,/IL/Chicago/1255-S-State-St-60605/unit-1617/ho...
6617,11824777,9137 S Paxton Ave,CHI - Calumet Heights,Single Family Residential,60617,"$239,000",list_agent_1,listing,Melanie Giglio,Melanie Giglio,...,0,48826241.56,0,48,48,0,0,32925.36587,239000,/IL/Chicago/9137-S-Paxton-Ave-60617/home/13115874
8816,11809180,4553 W 56th St #202,CHI - West Elsdon,Condo/Co-op,60629,"$170,000",list_agent_1,listing,Melanie Giglio,Melanie Giglio,...,0,32711091.43,0,62,62,0,0,26229.69560,170000,/IL/Chicago/4553-W-56th-St-60629/unit-202/home...
9022,11778614,4550 W 57th St Unit 3C4,CHI - West Elsdon,Condo/Co-op,60629,"$162,000",list_agent_1,listing,Melanie Giglio,Melanie Giglio,...,0,32711091.43,0,62,62,0,0,26229.69560,162000,/IL/Chicago/4550-W-57th-St-60629/unit-3C4/home...


### Galarza Team

url: https://thegalarzateam.com/

In [77]:
filtered_dealsheet.columns

Index(['mls_id', 'Address', 'Location', 'property_type', 'Zip_Code', 'Price',
       'agent_no', 'side', 'agent_name', 'Final_Agent/Team', 'Team_Members',
       'Brokerage', 'url', 'TRD_Note', 'geometry', 'SqFt', 'price_per_sq_ft',
       'year_built', 'broker_email', 'data_source_name', 'within_county',
       'Price_int', 'index_right', 'community', 'area', 'shape_area',
       'perimeter', 'area_num_1', 'area_numbe', 'comarea_id', 'comarea',
       'shape_len', 'price_as_int', 'api_url'],
      dtype='object')

In [128]:
dealsheet[dealsheet['agent_name'].str.contains("Lopez")].tail(60)

,mls_id,Address,Location,property_type,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,...,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,price_as_int,api_url
3141,11863767,3800 W 84th Pl,CHI - Ashburn,Single Family Residential,60652,"$325,000",buy_agent_1,buyer,Jonathan Lopez,Jonathan Lopez,...,0,1.354603e+08,0,70,70,0,0,54818.15463,325000,/IL/Chicago/3800-W-84th-Pl-60652/home/13967001
3398,11908148,3810 W 80th St,CHI - Ashburn,Single Family Residential,60652,"$315,750",list_agent_1,listing,Celeste Lopez,Celeste Lopez,...,0,1.354603e+08,0,70,70,0,0,54818.15463,315750,/IL/Chicago/3810-W-80th-St-60652/home/13967564
3639,11813312,4346 S Talman Ave,CHI - Brighton Park,Single Family Residential,60632,"$310,000",buy_agent_2,buyer,Saiva Lopez,Saiva Lopez,...,0,7.589279e+07,0,58,58,0,0,36537.13431,310000,/IL/Chicago/4346-S-Talman-Ave-60632/home/14035797
3804,12035679,5925 W 55 St,CHI - Garfield Ridge,Single Family Residential,60638,"$305,000",buy_agent_1,buyer,Margarita Lopez,Margarita Lopez,...,0,1.178908e+08,0,56,56,0,0,60080.44797,305000,/IL/Chicago/5925-W-55th-St-60638/home/14002489
3911,12016932,11000 S Avenue C,CHI - East Side,Single Family Residential,60617,"$300,800",list_agent_1,listing,Gustavo Lopez,Gustavo Lopez,...,0,8.324173e+07,0,52,52,0,0,52274.19285,300800,/IL/Chicago/11000-S-Avenue-C-60617/home/13034788
3993,11919301,5207 S Mason Ave,CHI - Garfield Ridge,Single Family Residential,60638,"$300,000",buy_agent_1,buyer,Magaly Lopez,Magaly Lopez,...,0,1.178908e+08,0,56,56,0,0,60080.44797,300000,/IL/Chicago/5207-S-Mason-Ave-60638/home/14026669
4028,11858069,3705 W 65th Pl,CHI - West Lawn,Single Family Residential,60629,"$300,000",list_agent_2,listing,Saiva Lopez,Saiva Lopez,...,0,8.231530e+07,0,65,65,0,0,41934.49013,300000,/IL/Chicago/3705-W-65th-Pl-60629/home/13990584
4036,12011381,5214 S Maplewood Ave,CHI - Gage Park,Single Family Residential,60632,"$300,000",buy_agent_1,buyer,Teresita Lopez,Teresita Lopez,...,0,6.128490e+07,0,63,63,0,0,32294.19591,300000,/IL/Chicago/5214-S-Maplewood-Ave-60632/home/14...
4220,12079140,7739 S Reilly Ave,CHI - Ashburn,Single Family Residential,60652,"$295,000",list_agent_1,listing,Vanessa Lopez,Vanessa Lopez,...,0,1.354603e+08,0,70,70,0,0,54818.15463,295000,/IL/Chicago/7739-S-Reilly-Ter-60652/home/22915585
4339,11775176,5716 S Artesian Ave,CHI - Gage Park,Single Family Residential,60629,"$290,000",buy_agent_1,buyer,Jonathan Lopez,Jonathan Lopez,...,0,6.128490e+07,0,63,63,0,0,32294.19591,290000,/IL/Chicago/5716-S-Artesian-Ave-60629/home/140...


In [88]:
Galarza_team.to_csv("Galarza_team.csv")

In [87]:
print(f"${Galarza_team['price_as_int'].sum():,}")

$9,218,300


### Meghan and John Healy

In [141]:
dealsheet[dealsheet['agent_name'].str.contains('')]

,mls_id,Address,Location,property_type,Zip_Code,Price,agent_no,side,agent_name,Final_Agent/Team,...,area,shape_area,perimeter,area_num_1,area_numbe,comarea_id,comarea,shape_len,price_as_int,api_url
2087,11925653,9146 S Oakley Ave,CHI - Beverly,Single Family Residential,60643,"$369,000",list_agent_1,listing,Michelle Dearing Shiels,Michelle Dearing Shiels,...,0,88779363.94,0,72,72,0,0,44591.4851,369000,/IL/Chicago/9146-S-Oakley-Ave-60643/home/13102259


In [148]:
healy_team = dealsheet[dealsheet['agent_name'].str.contains('Healy|Lauren Spencer|Shiels')]

In [150]:
print(f"${healy_team['price_as_int'].sum():,}")

$15,162,000


In [151]:
healy_team.to_csv("Healy_team.csv")

mls_id
11949831    2
11835861    2
11980120    2
11991471    2
11958204    1
11936516    1
12039691    1
11899243    1
11833806    1
11837916    1
11868857    1
11879457    1
11826688    1
11802111    1
11940403    1
11849944    1
11870799    1
12060679    1
11945598    1
11961773    1
11993499    1
12034102    1
12006044    1
11994952    1
11956606    1
11980147    1
11831820    1
11819207    1
11919349    1
12013262    1
11986759    1
12038403    1
11915558    1
Name: count, dtype: int64